# PISCES Wavelength calibration

In [1]:
import tools
import glob
from PISCESparams import Params
import numpy as np
import logging as log
from tools.initLogger import initLogger


## Load PISCES parameters

Most parameters are default already in params.py. Change the paths and names if necessary in the first part of the file.

In [2]:
par = Params()

# This is the logger system to print to file and to print to console
initLogger(par.exportDir+'/IFS.log')


## Define wavelengths and list of wavelength calibration files

Watch out, the two lists need to be in the same order. The default location of the calibration files is in par.wavecalDir='./ReferenceFiles/VARIAWavecal/'

In [3]:
par.lamlist = np.arange(605,725+1,10)
par.filelist = np.sort(glob.glob(par.wavecalDir + "Calibra*.FIT"))


## Generate wavelength calibration from scratch

This routine uses a first guess at the location of the lenslets, and then fits polynomials on the detector maps at each wavelength using an optimization routine. You can inspect the results in your folder par.wavecalDir as a set of .png images to make sure the wavelength calibration has succeeded.

The other files that are created with this routine: 
- lamsol.dat, which contains all the wavelengths and the coefficients of the polynomial representation of their positions
- PSFloc.fits, which contains the inverse wavelength solution consisting of the wavelength value at the center of each pixel for each micro-spectrum

In [4]:
from tools.wavecal import buildcalibrations

# these are the standard parameters for PISCES. You cannot use hires=True without risks.
# Parallelization is not used when hires=False
buildcalibrations(par,parallel=False,hires=False)

2017-02-02 17:32:08,129: INFO     Building calibration files, placing results in /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/VARIAwavecal/
2017-02-02 17:32:08,156: WARNING  Unexpected extra padding at the end of the file.  This padding may not be preserved when saving changes.
2017-02-02 17:32:08,166: INFO     Read data from HDU 0 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/VARIAwavecal/Calibra_161205_605nm.FIT
2017-02-02 17:32:08,172: INFO     Read data from HDU 0 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/VARIAwavecal/Calibra_161205_605nm.FIT
2017-02-02 17:32:09,278: INFO     Initializing PSFlet location transformation coefficients
2017-02-02 17:32:11,545: INFO     Performing initial optimization of PSFlet location transformation coefficients for frame /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/VARIAwavecal/Calibra_161205_605nm.FIT
2017-02-02 17:32:25,310: INFO     Performing final optimization of PSFlet location transformation coefficients for frame /Users/mrizzo

Total time elapsed: 673 s


# Reduce PISCES data

This is the standard reduction routine. This will work as long as the calibration files can be found within par.wavecalDir

The available methods for PISCES are:
- 'simple': sums pixel columns across the microspectrum
- 'dense': densifies the image and pixel columns across the microspectrum
- 'apphot': aperture photometry using photutils package to pixel columns across the microspectrum
- 'intopt': RECOMMENDED. Optimal extraction: uses inverse wavelength solution to fit gaussian values to each pixel column within the microspectrum, then remaps results on desired wavelength bins. There is no reason to use anything else than this in my opinion.

reduceIFSMap returns the IFS cube, and also creates a FITS file in par.exportDir with the same name as the input file, with an extension designating the method used.

There is more work to be done on this. We should create a mask of bad pixels and bad lenslets.

In [4]:
from IFS import reduceIFSMap
fname=par.filelist[0]    #INSERT YOUR PISCES FILENAME HERE
cube = reduceIFSMap(par,fname,method='intopt')


2017-02-02 17:52:50,128: WARNING  Unexpected extra padding at the end of the file.  This padding may not be preserved when saving changes.
2017-02-02 17:52:50,156: INFO     Read data from HDU 0 of /Users/mrizzo/IFS/IFS-Simulator/ReferenceFiles/VARIAwavecal/Calibra_161205_605nm.FIT
2017-02-02 17:52:50,173: INFO     Reduced cube will have 21 wavelength bins
tools/reduction.py:555: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  pix_center_vals = [np.sum(im.data[i1:i1 + delt_y, val]*gaussian) for val in ix]/weights
/Users/mrizzo/anaconda/envs/py27/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
2017-02-02 17:53:20,367: INFO     Writing data to /Users/mrizzo/IFS/IFS-Simulator/SimResults/Calibra_161205_605nm_red_intopt.fits


The list of wavelengths corresponding to the center and edges of each slice of the cube can be obtained by the following routine. It can be changed and tweaked by changing the parameter par.R

In [5]:
from tools.reduction import calculateWaveList
lam_midpts,lam_endpts = calculateWaveList(par)
print 

2017-02-02 17:54:00,038: INFO     Reduced cube will have 21 wavelength bins
